In [1]:
import pickle
import numpy as np
def save_streams(name, data):
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump(data, f)

def load_streams(name):
    with open(f'{name}.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

## ML

In [2]:
train_data = load_streams('NEW_TRAIN_DATA')
test_data = load_streams('NEW_TEST_DATA')

X_train, y_train = np.array(train_data['X_train']), np.array(train_data['y_train'])
y_train = np.array([d[0] for d in y_train])

X_test, y_test = np.array(test_data['X_test']), np.array(test_data['y_test'])
y_test = np.array([d[0] for d in y_test])

# print(len(X_train), len(y_train)) # 103 103
# print(X_train.shape, y_train.shape) # (103, 13, 100) (103,)

# print(len(X_test), len(y_test)) # 26 26
# print(X_test.shape, y_test.shape) # (16, 13, 100) (26,)

n_steps = X_train.shape[1]
n_features = X_train.shape[2]

In [3]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

2022-06-27 14:07:54.364678: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-27 14:07:54.364736: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:

# # define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
# fit model
model.fit(X_train, y_train, epochs=200, verbose=0, validation_data=(X_test, y_test))

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# demonstrate prediction
# x_input = np.array(X_test[0])
# print(x_input)
# x_input = x_input.reshape((1, n_steps, n_features))
# print(x_input)
# yhat = model.predict(x_input, verbose=0)
# print(yhat, y_test[0])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 80,501
Trainable params: 80,501
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 76.92%
